<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Managing-granulometry" data-toc-modified-id="Managing-granulometry-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Managing granulometry</a></span></li></ul></div>

# Managing granulometry

F. Dubois - 2016

Importing necessary modules

In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}
#%config InlineBackend.figure_formats = {'png', 'retina'}

import os,sys
import numpy
import math
import scipy
import scipy.special
import random
import matplotlib.pyplot as plt

# Seaborn, useful for graphics
#import seaborn as sns

# JB's favorite Seaborn settings for notebooks
#rc = {'lines.linewidth': 2, 
#      'axes.labelsize': 18, 
#      'axes.titlesize': 18, 
#      'axes.facecolor': 'DFDFE5'}
#sns.set_context('notebook', rc=rc)
#sns.set_style('darkgrid', rc=rc)


# Import Bokeh modules for interactive plotting
#import bokeh.io
#import bokeh.mpl
#import bokeh.plotting
#bokeh.io.output_notebook()

from pylmgc90 import pre


A function to draw the cumulated volume of particles

In [ ]:
def draw_granulo(radii):
    """
    A function to draw the cumulative volume occupied by a list of particles with respect to the radius 
   
    :param radii: an array of radii
    """  
    
    rd = numpy.sort(radii)

    # computing total volume (4pi/3 voluntarily omitted)
    vv=0.
    for radius in radii:
        vv+=math.pow(radius,3)

    # ecriture granulo  
    v=0.
    radius_prev=rd[0]
    x=[]
    y=[]  
    for radius in rd:
        v += math.pow(radius,3)
        if radius != radius_prev:
            x.append(radius_prev)
            y.append(v/vv)

        radius_prev = radius

        x.append(radius_prev)
        y.append(v/vv)

    plt.plot(numpy.array(x),numpy.array(y))
    plt.xlabel('radius')
    plt.ylabel('cumulated volume/total volume')
    plt.title('Cumulated volume')
    plt.grid(True)
    #plt.savefig("test.png")
    plt.show()
    # Make it interactive with Bokeh
    #bokeh.plotting.show(bokeh.mpl.to_bokeh())

Definition of a distribution function approximated using $\beta(x;a,b) = \frac{\Gamma(a+b)}{\Gamma(a)*\Gamma(b)} \int_0^x (t^{a-1} (1-t)^{b-1} \mathrm{d}t $ where $\Gamma(x) = \int_0^\infty t^{x-1} e^{-t} \mathrm{d}t$, see Voivret. 

In [ ]:
def fuseau(Rmin,Rmax,V,a,b):
    """
    A function to compute a radius distribution in a given volume 
   
    :param Rmin: min radius
    :param Rmax: max radius
    :param V   : total volume
    :param a,b : two parameters describing the function
    :return    : a list of radii
    """  

    #####################

    # number of subdivisions for the discretization of the curve  
    T = 10.

    # x between 0 et 1
    x=0.

    # adimensionning volume
    vv = V/(4.*math.pi/3.)

    # computing the radius range of each 
    dr= (Rmax-Rmin)/T
    r_b= Rmin

    radii = numpy.array([],'d')

    k= numpy.array(range(1, int(T)+1, 1),'d')
    k= k/T

    vv_beg=0.
    for x in k:
        beta= scipy.special.betainc(a,b,x)

        # computing rmin and rmax of each subinterval
        r_e= r_b + dr
        r = 0.5*(r_b+r_e)
        #r= dr*x + r_b

        # evaluating the number of grains for each subinterval
        n = ((beta*vv)-vv_beg)/(r*r*r)

        # random distribution in the subinterval [r_b, r_e]
        radii_interval=pre.granulo_Random(int(n), r_b, r_e)

        # adding radii to the list
        radii= numpy.concatenate( (radii_interval,radii) )
    
        # updating for next loop
        r_b= r_e
        vv_beg=beta*vv 
    # 
    return radii

Lets play with existing functions

In [ ]:
#Rmin and Rmax are the minimal and maximal radii of the generated grains
Rmin= 4.e-3
Rmax= 1.e-2

# Total volume of grains
V= 0.1

# beta computation
#(computation of the primitive and replacement of x)
# input parameters
a= 3.
b= 1.

# home made
radii = fuseau(Rmin,Rmax,V,a,b)

# following a uniform distribution in number
#radii = pre.granulo_Random(1000, Rmin, Rmax)

# following a uniform distribution in surface
#radii = pre.granulo_Uniform(1000, Rmin, Rmax)

# following a binomial distribution in number
#radii = pre.granulo_TwoSizesNumber(1000, Rmin, Rmax,0.2)

draw_granulo(radii) 
